# Exchange Rates
This script pulls forex data from the European Central Bank via the exchangerate.host platform. Data goes as far back as 1 Jan. 2010. 

If you have questions please email charles@cmefi.com or jasmine@cmefi.com

# I. Preparation: Install Libraries, Set Variables

In [1]:
# Install/import relevant libraries
import requests 
import datetime
import json
import os
import smtplib
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound

In [3]:
# Set variables of interest. Namely, start and end dates, base currency, currencies of interest,
# and forex rate data source. 

# Start and end dates for forex time series 
start = str(datetime.date(2010, 1, 1).strftime('%Y-%m-%d'))
end = str(datetime.date.today())

# Base currency 
base = 'USD'

# Currencies of interest 
currencies = ['CAD', 'EUR', 'INR', 'MXN', 'CNY', 'TWD']

# Forex rate data source 
fx_source = 'ecb' # This is the European Central Bank 

# Delimiter 
s = ','

In [1]:
# Get request 
url = 'https://api.exchangerate.host/timeseries?' + \
'&start_date=' + start + \
'&end_date=' + end + \
'&base=' + base + \
'&symbols=' + s.join(currencies) + \
'&source=' + fx_source

# Check full URL
print(url)

NameError: name 'start' is not defined

# II. Parse: Obtain and Structure Forex Rate Data 
After submitting the get request, we obtain the desired forex time series as a JSON object - this is then parsed and converted to a pandas df for further use. 

In [5]:
# Pull forex rates 
rates = requests.get(url) 
rate_data = rates.json()

In [6]:
# Convert from json/python object --> json string --> python dictionary 
rate_string = json.dumps(rate_data)
rate_list = json.loads(rate_string)['rates']

# Restructure dataframe 
rate_df_initial = pd.DataFrame.from_records(rate_list)
rate_df_initial['currency'] = rate_df_initial.index
rate_df = rate_df_initial.melt(
    id_vars = ['currency'],
    value_vars = None,
    var_name = 'date',
    value_name = 'value'
)

# Check 
print(rate_df)


     currency        date      value
0         CAD  2010-01-01   1.050118
1         CNY  2010-01-01   6.827017
2         INR  2010-01-01  46.536165
3         MXN  2010-01-01  13.135013
4         EUR  2010-01-01   0.694155
...       ...         ...        ...
1825      CAD  2011-01-01   0.997006
1826      CNY  2011-01-01   6.602305
1827      INR  2011-01-01  44.722347
1828      MXN  2011-01-01  12.383999
1829      EUR  2011-01-01   0.748391

[1830 rows x 3 columns]


In [17]:
# Alternative method of creating dataframe 

'''
rate_df = pd.DataFrame(columns = ['date', 'currency', 'value'])
print(rate_df)

for d in rate_list:
    # print(d)
    # print(rate_list[d])
    for currency in rate_list[d]:
        # print(currency)
        rate_dict = {
            'date': d,
            'currency': currency,
            'value': rate_list[d][currency]
            }
        # print(pd.DataFrame(data = rate_dict))
        rate_df = rate_df.append(rate_dict, ignore_index = True)
        # print(rate_list[d][currency])
        
print(rate_df)

'''

"\nrate_df = pd.DataFrame(columns = ['date', 'currency', 'value'])\nprint(rate_df)\n\nfor d in rate_list:\n    # print(d)\n    # print(rate_list[d])\n    for currency in rate_list[d]:\n        # print(currency)\n        rate_dict = {\n            'date': d,\n            'currency': currency,\n            'value': rate_list[d][currency]\n            }\n        # print(pd.DataFrame(data = rate_dict))\n        rate_df = rate_df.append(rate_dict, ignore_index = True)\n        # print(rate_list[d][currency])\n        \nprint(rate_df)\n\n"